# Trace test

Estudio de la traza de una intersección sencilla gestionada por un semáforo. 

Road 0 -> Road 3 

Road 1 -> Road 4 

In [82]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import nbformat

Lectura de trazas

In [63]:
def process_trace(filename, t_max):
    """
    Processes a CSV trace file of a road generated by flowsync2 containing
    time-stamped 'push' and 'get' actions.

    This function reads the trace from a CSV file, filters out entries beyond a
    specified time window t_max, and computes a
    running count of 'push' minus 'get' actions over time.

    Parameters:
        filename (str): Path to the CSV file containing the trace.
        t_max (float): Maximum time to include in the trace.

    Returns:
        tuple:
            - df (DataFrame): Original data from the CSV file.
            - ac_df (DataFrame): Time series of action counts with columns ['time', 'ac'].
    """
    df = pd.read_csv(filename, index_col=False)
    df_sorted = df.sort_values(by='time').reset_index(drop=True)

    df_sorted = df_sorted[df_sorted.time <=t_max]

    ac = 0
    ac_list = []

    for _, row in df_sorted.iterrows():
        if row['action'] == 'push':
            ac += 1
        elif row['action'] == 'get':
            ac -= 1
        ac_list.append({'time': row['time'], 'ac': ac})

    ac_df = pd.DataFrame(ac_list)

    return df, ac_df


In [77]:
def resume_per_phase(df):

    # Ensure 'time' is float
    df['time'] = df['time'].astype(float)

    # Create 45-second interval bin index (0, 1, 2, ...)
    df['time_group'] = (df['time'] // 45).astype(int)

    # Group by interval number and action type
    grouped = df.groupby(['time_group', 'action']).size().reset_index(name='count')

    # Pivot to separate push/get
    pivoted = grouped.pivot(index='time_group', columns='action', values='count').fillna(0).astype(int).reset_index()
    pivoted.drop("action")
    return pivoted



In [90]:
road0, road0_ac = process_trace("road0.csv", 1800)
road3, road3_ac = process_trace("road3.csv", 1800)

road1, road1_ac = process_trace("road1.csv", 1800)
#road4, road4_ac = process_trace("road4.csv", 1800)

intersec0, intersec_ac0 = process_trace("intersection0.csv", 1800)
#intersec1, intersec_ac1 = process_trace("intersection1.csv", 1800)


In [78]:
res_road0 = resume_per_phase(road0)
res_road1 = resume_per_phase(road1)


KeyError: "['action'] not found in axis"

In [91]:
res_road0.head(20)


action,time_group,get,push
0,0,26,40
1,2,45,46
2,4,43,44
3,6,45,43
4,7,0,1
5,8,45,44
6,10,43,43
7,12,43,44
8,14,42,41
9,16,44,46


In [68]:

def plot_trace(road0_ac, road3_ac, intersec_ac, road0_name, road3_name):
    fig = make_subplots()

    fig.add_trace(go.Scatter(
        x = road0_ac['time'],
        y = road0_ac['ac'],
        mode ='lines+markers',
        name = road0_name
    ))

    fig.add_trace(go.Scatter(
        x = road3_ac['time'],
        y = road3_ac['ac'],
        mode='lines+markers',
        name=road3_name
    ))

    fig.add_trace(go.Scatter(
        x = intersec_ac['time'],
        y = intersec_ac['ac'],
        mode='lines+markers',
        name="intersection"
    ))

    fig.update_layout(
        title='AC over Time',
        xaxis_title='Time',
        yaxis_title='AC Value',
        hovermode='closest',
        width=1200,
        height=400
    )

    fig.show()


In [88]:
road0

,id_car,time,action,inQ,outQ,outinQ
0,29113,29610.775577,get,43,28,26
1,29170,29610.775577,push,43,28,27
2,29114,29612.111056,get,44,29,27
3,29172,29612.111056,push,44,29,28
4,29115,29613.202869,get,45,30,28
...,...,...,...,...,...,...
6335,35423,36012.004845,get,5,1,88
6336,35424,36012.781748,get,5,2,88
6337,35425,36013.831836,get,5,3,88
6338,35427,36014.747995,get,5,4,88


In [92]:
plot_trace(road0_ac, road3_ac,intersec_ac0, "Road 0", "Road 3")

In [ ]:
plot_trace(road1_ac, road0_ac,intersec_ac0, "Road 1", "Road 0")

: 

In [71]:
intersec_ac.head(20)

,time,ac
0,9.000000,1
1,10.309091,0
2,18.000000,1
3,18.856590,2
4,19.309091,1
5,19.511699,2
6,20.165681,1
7,20.641794,2
8,20.820790,1
9,21.559093,2


In [72]:
import pygame
import time

In [73]:
import pygame
import pandas as pd
import time
import cv2
import numpy as np

road0_small, road0_ac_small = process_trace("road0.csv", 15)

# Convert to car entry/exit intervals
entries = road0_small[road0_small['action'] == 'push'].set_index('id_car')
exits = road0_small[road0_small['action'] == 'get'].set_index('id_car')

car_data = pd.DataFrame({
    'entry': entries['time'],
    'exit': exits['time']
})
car_data = car_data.reset_index()

# Pygame setup
WIDTH, HEIGHT = 800, 200
street_start_x = 100
street_end_x = WIDTH - 100
car_width = 50
fps = 30
duration = car_data['exit'].max() + 2  # total sim time

# Initialize Pygame (off-screen rendering for video)
pygame.init()
screen = pygame.Surface((WIDTH, HEIGHT))
font = pygame.font.SysFont(None, 24)

# Set up video writer
video = cv2.VideoWriter('street_simulation.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (WIDTH, HEIGHT))

def draw_frame(current_time):
    screen.fill((240, 240, 240))

    # Draw street
    pygame.draw.rect(screen, (200, 200, 200), (street_start_x, 80, street_end_x - street_start_x, 40))

    # Draw each car based on current time
    for _, car in car_data.iterrows():
        if car['entry'] <= current_time <= car['exit']:
            progress = (current_time - car['entry']) / (car['exit'] - car['entry'])
            x = street_start_x + (1 - progress) * (street_end_x - street_start_x - car_width)
            pygame.draw.rect(screen, (30, 144, 255), (x, 85, car_width, 30))
            label = font.render(f"{int(car['id_car'])}", True, (255, 255, 255))
            screen.blit(label, (x + 15, 90))

    # Clock
    time_label = font.render(f"Time: {current_time:.2f}", True, (0, 0, 0))
    screen.blit(time_label, (10, 10))

    # Convert to OpenCV frame and write to video
    frame = pygame.surfarray.array3d(screen)
    frame = frame.transpose([1, 0, 2])  # Pygame -> OpenCV format
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    video.write(frame)

# Render frames
num_frames = int(duration * fps)
for frame_num in range(num_frames):
    sim_time = frame_num / fps
    draw_frame(sim_time)

video.release()
pygame.quit()


KeyboardInterrupt: 